In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

In [2]:
import sys
python = sys.executable

# as well as install a language model:
!{python} -m spacy download en_core_web_md
!{python} -m spacy link en_core_web_md en --force;

symbolic link created for C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\spacy\data\en_core_web_md <<===>> C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\en_core_web_md

    Linking successful
    C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\en_core_web_md
    -->
    C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\spacy\data\en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



usage: spacy link [-h] [-f] origin link_name [model_path]
spacy link: error: unrecognized arguments: --force;


In [3]:
import rasa_nlu
import rasa_core
import spacy

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("en")("Hello world!"))

rasa_nlu: 0.11.5 rasa_core: 0.10.3
Loading spaCy language model...
Hello world!


In [4]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- I am going to save the world
- super
- extremely good
- so so perfect
- so good
- so perfect

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- extremly sad
- so saad
- Quite bad - can I get a cute picture of a [bird](group:birds), please?
- Really bad and only [doggo](group:shibes) pics and change that.
- Not good. The only thing that could make me fell better is a picture of a cute [kitten](group:cats).
- so sad. Only the picture of a [puppy](group:shibes) could make it better.
- I am very sad. I need a [cat](group:cats) picture.
- Extremely sad. Only the cute [doggo](group:shibes) pics can make me feel better.
- Bad. Please show me a [bird](group:birds) pic!
- Pretty bad to be honest. Can you show me a [puppy](group:shibes) picture to make me fell better?

## intent: inform
- A [dog](group:shibes)
- [dog](group:shibes)
- [bird](group:birds)
- a [cat](group:cats)
- [cat](group:cats)
- a [bird](group:birds)
- of a [dog](group:shibes)
- of a [cat](group:cats)
- a [bird](group:birds), please
- a [dog](group:shibes), please
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [5]:
import yaml


In [6]:
config = """
{
"language": "en", 
"pipeline": "spacy_sklearn"
}
"""

%store config > config.json

Writing 'config' (str) to file 'config.json'.


In [7]:
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

# loading the nlu training samples
training_data = load_data('nlu.md')

# trainer to educate our pipeline
trainer = Trainer(RasaNLUConfig('config.json'))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.converters:Training data format at nlu.md is md
INFO:rasa_nlu.training_data:Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- found intents: 'goodbye', 'greet', 'inform', 'mood_affirm', 'mood_deny', 'mood_great', 'mood_unhappy'
	- entity examples: 18 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:rasa_nlu.model:Starting to train component nlp_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component tokenizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_entity_featurizer_regex
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train co

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.1s finished
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into 'C:\Users\Home\nlp_combined_project\models\nlu\default\current'


In [8]:
print(interpreter.parse("I am sad, plased send me a picture of a dog"))

C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'intent': {'name': 'mood_unhappy', 'confidence': 0.4920102315915817}, 'entities': [{'start': 40, 'end': 43, 'value': 'shibes', 'entity': 'group', 'extractor': 'ner_crf', 'processors': ['ner_synonyms']}], 'intent_ranking': [{'name': 'mood_unhappy', 'confidence': 0.4920102315915817}, {'name': 'goodbye', 'confidence': 0.15873745561378058}, {'name': 'mood_great', 'confidence': 0.10669776342992669}, {'name': 'inform', 'confidence': 0.09362963032213877}, {'name': 'greet', 'confidence': 0.08047219337239792}, {'name': 'mood_affirm', 'confidence': 0.048244695375581265}, {'name': 'mood_deny', 'confidence': 0.02020803029459333}], 'text': 'I am sad, plased send me a picture of a dog'}


In [9]:
rasa_nlu.__version__

'0.11.5'

In [10]:
rasa_core.__version__

'0.10.3'

In [12]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
print("-----------------------------------------------------------------\n\n")
while True:
    
    print("\n\n user: ") 
    a = input()
    if a == 'stop':
        break
        
    print("\n\n chatbot agent: ") 
    responses = input()
    

Your bot is ready to talk! Type your messages here or send 'stop'
-----------------------------------------------------------------




 user: 
hi


 chatbot agent: 
hello


 user: 
stop


In [13]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

In [14]:
# Classification models in DeepPavlov

from deeppavlov import build_model, configs

CONFIG_PATH = configs.classifiers.intents_snips

model = build_model(CONFIG_PATH, download=True)

print(model(["What is the weather in Boston today?"]))

2019-03-06 06:46:27.947 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-03-06 06:46:29.104 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /datasets/snips_intents/train.csv.md5 HTTP/1.1" 200 44
2019-03-06 06:46:29.151 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/datasets/snips_intents/train.csv download because of matching hashes
2019-03-06 06:46:29.166 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-03-06 06:46:29.557 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/embeddings/dstc2_fastText_model.bin.md5 HTTP/1.1" 200 59
2019-03-06 06:46:29.573 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-03-06 

__________________________________________________________________________________________________


[['GetWeather']]


In [15]:
import tensorflow
tensorflow.Session().close()

tensorflow.reset_default_graph()

In [16]:
import gc
gc.collect()

5880

In [17]:
# Classification models in DeepPavlov

from deeppavlov import build_model, configs

#CONFIG_PATH = configs.classifiers.intents_snips


gc.collect()
model = build_model("intents_snips.json")

print(model(["What is the weather in Boston today?"]))





2019-03-06 06:51:22.969 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from C:\Users\Home\.deeppavlov\models\classifiers\intents_snips_v9\classes.dict]
2019-03-06 06:51:22.984 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `C:\Users\Home\.deeppavlov\downloads\embeddings\dstc2_fastText_model.bin`]
2019-03-06 06:52:39.819 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 303: [initializing `KerasClassificationModel` from saved]
2019-03-06 06:52:41.695 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 313: [loading weights from model.h5]
2019-03-06 06:52:47.804 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 137: Model was successfully initialized!
Model summary:
__________________________________________

[['GetWeather']]


In [18]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

In [19]:
hello = PatternMatchingSkill(responses=['Hello world!'], patterns=["hi", "hello", "good day"])
bye = PatternMatchingSkill(['Goodbye world!', 'See you around'],
                           patterns=["bye", "chao", "see you"])
fallback = PatternMatchingSkill(["I don't understand, sorry", 'I can say "Hello world!"'])

In [20]:
agent = DefaultAgent([hello, bye, fallback], skills_selector=HighestConfidenceSelector())

In [21]:
agent(['Hello', 'Bye', 'I can say "Hello world!"'])

['Hello world!', 'See you around', 'Hello world!']

In [22]:
agent(['Hello', 'Bye', 'Or not'])

['Hello world!', 'See you around', 'I can say "Hello world!"']

In [23]:
gc.collect()

from deeppavlov import build_model, configs
model = build_model("morpho_en.json")

2019-03-06 06:55:48.785 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from C:\Users\Home\.deeppavlov\models\morpho_tagger\UD2.0\en\tag.dict]
2019-03-06 06:55:48.847 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from C:\Users\Home\.deeppavlov\models\morpho_tagger\UD2.0\en\char.dict]
2019-03-06 06:55:48.941 INFO in 'deeppavlov.models.morpho_tagger.network'['network'] at line 126: 82 symbols, 119 tags in CharacterTagger
2019-03-06 06:55:50.144 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 106: __________________________________________________________________________________________________
2019-03-06 06:55:50.160 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104: Layer (type)                    Output Shape         Param #     Connected to                     
2019-03-06 06:55:50.176 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 108: =========

In [24]:
sentences = ["I was walking home on an unfamiliar street.", "The girl sang in the church choir about all tired in a strange land."]
for parse in model(sentences):
    print(parse)

1	I	PRON	Case=Nom|Number=Sing|Person=1|PronType=Prs
2	was	AUX	Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin
3	walking	VERB	Tense=Pres|VerbForm=Part
4	home	ADV	_
5	on	ADP	_
6	an	DET	Definite=Ind|PronType=Art
7	unfamiliar	ADJ	Degree=Pos
8	street	NOUN	Number=Sing
9	.	PUNCT	_

1	The	DET	Definite=Def|PronType=Art
2	girl	NOUN	Number=Sing
3	sang	NOUN	Number=Sing
4	in	ADP	_
5	the	DET	Definite=Def|PronType=Art
6	church	NOUN	Number=Sing
7	choir	NOUN	Number=Sing
8	about	ADP	_
9	all	ADV	_
10	tired	ADJ	Degree=Pos
11	in	ADP	_
12	a	DET	Definite=Ind|PronType=Art
13	strange	ADJ	Degree=Pos
14	land	NOUN	Number=Sing
15	.	PUNCT	_



In [25]:
sentences = [["I", "was", "walking", "home", "on", "an", "unfamiliar", "street", "."]]
for parse in model(sentences):
    print(parse)

1	I	PRON	Case=Nom|Number=Sing|Person=1|PronType=Prs
2	was	AUX	Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin
3	walking	VERB	Tense=Pres|VerbForm=Part
4	home	ADV	_
5	on	ADP	_
6	an	DET	Definite=Ind|PronType=Art
7	unfamiliar	ADJ	Degree=Pos
8	street	NOUN	Number=Sing
9	.	PUNCT	_



In [26]:
from deeppavlov.core.commands.utils import parse_config
config_dict = parse_config(configs.ner.ner_ontonotes)
print(config_dict['dataset_reader']['data_path'])

~/.deeppavlov/downloads/ontonotes


In [27]:

raw_corpus = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

%store raw_corpus > mycorpus.txt

Writing 'raw_corpus' (list) to file 'mycorpus.txt'.


In [28]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
processed_corpus

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [30]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
dictionary.save(os.path.join("TEMP_FOLDER", 'deerwester.dict'))  # store the dictionary, for future reference

print(dictionary)

2019-03-06 07:06:48.520 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 202: adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 07:06:48.523 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 209: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2019-03-06 07:06:48.527 INFO in 'gensim.utils'['utils'] at line 542: saving Dictionary object under TEMP_FOLDER\deerwester.dict, separately None
2019-03-06 07:06:48.531 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'TEMP_FOLDER\\deerwester.dict'}
2019-03-06 07:06:48.576 INFO in 'gensim.utils'['utils'] at line 556: saved TEMP_FOLDER\deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [31]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [32]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(0, 1), (1, 1)]

In [34]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
print(bow_corpus)
print("\n------------------------------------------------\n\n")

corpora.MmCorpus.serialize(os.path.join("TEMP_FOLDER1", 'deerwester.mm'), bow_corpus)  # store to disk, for later use

for c in bow_corpus:
    print(c)

[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]

------------------------------------------------




2019-03-06 07:08:53.457 INFO in 'gensim.corpora.mmcorpus'['mmcorpus'] at line 122: storing corpus in Matrix Market format to TEMP_FOLDER1\deerwester.mm
2019-03-06 07:08:53.460 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb+', 'uri': 'TEMP_FOLDER1\\deerwester.mm'}
2019-03-06 07:08:53.464 INFO in 'gensim.matutils'['matutils'] at line 1244: saving sparse matrix to TEMP_FOLDER1\deerwester.mm
2019-03-06 07:08:53.468 INFO in 'gensim.matutils'['matutils'] at line 1356: PROGRESS: saving document #0
2019-03-06 07:08:53.472 INFO in 'gensim.matutils'['matutils'] at line 1376: saved 9x12 matrix, density=25.926% (28/108)
2019-03-06 07:08:53.485 DEBUG in 'gensim.matutils'['matutils'] at line 1398: closing TEMP_FOLDER1\deerwester.mm
2019-03-06 07:08:53.489 DEBUG in 'gensim.matutils'['matutils'] at line 1398: closing TEMP_FOLDER1\deerwester.mm
2019-03-06 07:08:53.492 INFO in 'gensim.corpora.indexedcorpus'['indexedcorpus'] at line 135: saving MmCorpus index t

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [35]:
from gensim import models
# train the model
tfidf = models.TfidfModel(bow_corpus)
# transform the "system minors" string
tfidf[dictionary.doc2bow("system minors".lower().split())]

2019-03-06 07:09:13.449 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 378: collecting document frequencies
2019-03-06 07:09:13.457 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 384: PROGRESS: processing document #0
2019-03-06 07:09:13.460 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 396: calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


[(5, 0.5898341626740045), (11, 0.8075244024440723)]

In [36]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [37]:
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "C:\Users\Home\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [38]:
from smart_open import smart_open
class MyCorpus(object):
    def __iter__(self):
        for line in smart_open('mycorpus.txt', 'rb'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [39]:

corpus_memory_friendly = MyCorpus() # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [40]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

2019-03-06 07:09:26.642 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'mycorpus.txt'}


[(0, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (7, 1)]
[(2, 1), (7, 1), (8, 1)]
[(1, 1), (5, 1)]
[(3, 1), (6, 1), (7, 1)]
[]
[(10, 1)]
[(9, 1), (11, 1)]
[(11, 1)]


In [41]:
from six import iteritems
from smart_open import smart_open

# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in smart_open('mycorpus.txt', 'rb'))

# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

# remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids + once_ids)
print(dictionary)

2019-03-06 07:09:31.603 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'mycorpus.txt'}
2019-03-06 07:09:31.608 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 202: adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 07:09:31.612 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 209: built Dictionary(51 unique tokens: ["['human", 'abc', "applications',", 'computer', 'for']...) from 9 documents (total 69 corpus positions)
2019-03-06 07:09:31.616 DEBUG in 'gensim.corpora.dictionary'['dictionary'] at line 461: rebuilding dictionary, shrinking gaps


Dictionary(9 unique tokens: ['computer', 'interface', 'response', 'system', 'user']...)


In [42]:
# create a toy corpus of 2 documents, as a plain Python list
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.mm'), corpus)

2019-03-06 07:09:44.900 INFO in 'gensim.corpora.mmcorpus'['mmcorpus'] at line 122: storing corpus in Matrix Market format to C:\Users\Home\AppData\Local\Temp\corpus.mm
2019-03-06 07:09:44.909 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb+', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.mm'}
2019-03-06 07:09:45.202 INFO in 'gensim.matutils'['matutils'] at line 1244: saving sparse matrix to C:\Users\Home\AppData\Local\Temp\corpus.mm
2019-03-06 07:09:45.205 INFO in 'gensim.matutils'['matutils'] at line 1356: PROGRESS: saving document #0
2019-03-06 07:09:45.214 INFO in 'gensim.matutils'['matutils'] at line 1376: saved 2x2 matrix, density=25.000% (1/4)
2019-03-06 07:09:45.280 DEBUG in 'gensim.matutils'['matutils'] at line 1398: closing C:\Users\Home\AppData\Local\Temp\corpus.mm
2019-03-06 07:09:45.286 DEBUG in 'gensim.matutils'['matutils'] at line 1398: closing C:\Users\Home\AppData\Local\Temp\corpus.mm
2019-03-06 07:09:45.384 INFO in 'ge

In [44]:
corpora.SvmLightCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.svmlight'), corpus)
corpora.BleiCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.lda-c'), corpus)
corpora.LowCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.low'), corpus)

2019-03-06 07:10:53.740 INFO in 'gensim.corpora.svmlightcorpus'['svmlightcorpus'] at line 112: converting corpus to SVMlight format: C:\Users\Home\AppData\Local\Temp\corpus.svmlight
2019-03-06 07:10:53.747 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.svmlight'}
2019-03-06 07:10:53.752 INFO in 'gensim.corpora.indexedcorpus'['indexedcorpus'] at line 135: saving SvmLightCorpus index to C:\Users\Home\AppData\Local\Temp\corpus.svmlight.index
2019-03-06 07:10:53.755 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.svmlight.index'}
2019-03-06 07:10:53.759 INFO in 'gensim.corpora.bleicorpus'['bleicorpus'] at line 143: no word id mapping provided; initializing from corpus
2019-03-06 07:10:53.763 INFO in 'gensim.corpora.bleicorpus'['bleicorpus'] at line 151: storing corpus in Blei's LDA-C format int

In [45]:
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'corpus.mm'))

2019-03-06 07:11:03.639 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.mm.index'}
2019-03-06 07:11:03.649 INFO in 'gensim.corpora.indexedcorpus'['indexedcorpus'] at line 67: loaded corpus index from C:\Users\Home\AppData\Local\Temp\corpus.mm.index
2019-03-06 07:11:03.653 INFO in 'gensim.corpora._mmreader'['mmcorpus'] at line 66: initializing cython corpus reader from C:\Users\Home\AppData\Local\Temp\corpus.mm
2019-03-06 07:11:03.657 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.mm'}
2019-03-06 07:11:03.660 INFO in 'gensim.corpora._mmreader'['mmcorpus'] at line 66: accepted corpus with 2 documents, 2 features, 1 non-zero entries


In [46]:
print(corpus)

MmCorpus(2 documents, 2 features, 1 non-zero entries)


In [47]:
# one way of printing a corpus: load it entirely into memory
print(list(corpus))  # calling list() will convert any sequence to a plain Python list

2019-03-06 07:11:05.503 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.mm'}


[[(1, 0.5)], []]


In [48]:
# another way of doing it: print one document at a time, making use of the streaming interface
for doc in corpus:
    print(doc)

2019-03-06 07:11:06.309 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.mm'}


[(1, 0.5)]
[]


In [49]:
corpora.BleiCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.lda-c'), corpus)

2019-03-06 07:11:18.874 INFO in 'gensim.corpora.bleicorpus'['bleicorpus'] at line 143: no word id mapping provided; initializing from corpus
2019-03-06 07:11:18.883 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.mm'}
2019-03-06 07:11:18.886 INFO in 'gensim.corpora.bleicorpus'['bleicorpus'] at line 151: storing corpus in Blei's LDA-C format into C:\Users\Home\AppData\Local\Temp\corpus.lda-c
2019-03-06 07:11:18.890 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.lda-c'}
2019-03-06 07:11:18.894 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\corpus.mm'}
2019-03-06 07:11:18.898 INFO in 'gensim.corpora.bleicorpus'['bleicorpus'] at line 162: saving vocabulary of 2 words to C:\Users\Home\AppData\Local\Temp\

Compatibility with NumPy and SciPy

In [50]:

import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5,2])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix_dense = gensim.matutils.corpus2dense(corpus, num_terms=10)

In [51]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)

other

In [52]:
from gensim import corpora, models, similarities
if os.path.isfile(os.path.join(TEMP_FOLDER, 'deerwester.dict')):
    dictionary = corpora.Dictionary.load(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
    corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

2019-03-06 07:11:31.591 INFO in 'gensim.utils'['utils'] at line 422: loading Dictionary object from C:\Users\Home\AppData\Local\Temp\deerwester.dict
2019-03-06 07:11:31.599 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.dict'}
2019-03-06 07:11:31.680 INFO in 'gensim.utils'['utils'] at line 428: loaded C:\Users\Home\AppData\Local\Temp\deerwester.dict
2019-03-06 07:11:31.686 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm.index'}
2019-03-06 07:11:31.811 INFO in 'gensim.corpora.indexedcorpus'['indexedcorpus'] at line 67: loaded corpus index from C:\Users\Home\AppData\Local\Temp\deerwester.mm.index
2019-03-06 07:11:31.821 INFO in 'gensim.corpora._mmreader'['mmcorpus'] at line 66: initializing cython corpus reader from C:\Users\Home\AppData\Local\Temp\deerwester.mm
2019-03-06 07:11:31

Used files generated from first tutorial


In [53]:
print(dictionary[0])
print(dictionary[1])
print(dictionary[2])

computer
human
interface


In [54]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [55]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

2019-03-06 07:11:43.666 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}


[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [56]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2019-03-06 07:12:05.939 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 420: using serial LSI version on this node
2019-03-06 07:12:05.947 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 466: updating model with new documents
2019-03-06 07:12:05.951 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}
2019-03-06 07:12:05.956 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 492: preparing a new chunk of documents
2019-03-06 07:12:05.961 DEBUG in 'gensim.models.lsimodel'['lsimodel'] at line 496: converting corpus to csc format
2019-03-06 07:12:05.968 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 912: using 100 extra samples and 2 power iterations
2019-03-06 07:12:05.972 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 920: 1st phase: constructing (12, 102) action matrix
2019-03-06 07:12:06.59 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 935: orthonormali

In [57]:
lsi.print_topics(2)

2019-03-06 07:12:15.701 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 704: topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2019-03-06 07:12:15.706 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 704: topic #1(1.476): 0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"')]

In [58]:

for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

2019-03-06 07:12:17.608 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}


[(0, 0.0660078339609049), (1, 0.5200703306361851)]
[(0, 0.1966759285914263), (1, 0.7609563167700037)]
[(0, 0.08992639972446606), (1, 0.7241860626752508)]
[(0, 0.07585847652178314), (1, 0.6320551586003431)]
[(0, 0.10150299184980194), (1, 0.5737308483002946)]
[(0, 0.7032108939378305), (1, -0.16115180214025862)]
[(0, 0.8774787673119825), (1, -0.16758906864659537)]
[(0, 0.9098624686818572), (1, -0.14086553628719187)]
[(0, 0.6165825350569278), (1, 0.053929075663892156)]


In [59]:
lsi.save(os.path.join(TEMP_FOLDER, 'model.lsi')) # same for tfidf, lda, ...
#lsi = models.LsiModel.load(os.path.join(TEMP_FOLDER, 'model.lsi'))

2019-03-06 07:12:36.938 INFO in 'gensim.utils'['utils'] at line 542: saving Projection object under C:\Users\Home\AppData\Local\Temp\model.lsi.projection, separately None
2019-03-06 07:12:36.947 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\model.lsi.projection'}
2019-03-06 07:12:37.0 INFO in 'gensim.utils'['utils'] at line 556: saved C:\Users\Home\AppData\Local\Temp\model.lsi.projection
2019-03-06 07:12:37.5 INFO in 'gensim.utils'['utils'] at line 542: saving LsiModel object under C:\Users\Home\AppData\Local\Temp\model.lsi, separately None
2019-03-06 07:12:37.11 INFO in 'gensim.utils'['utils'] at line 648: not storing attribute projection
2019-03-06 07:12:37.17 INFO in 'gensim.utils'['utils'] at line 648: not storing attribute dispatcher
2019-03-06 07:12:37.21 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\

In [60]:
model = models.TfidfModel(corpus, normalize=True)

2019-03-06 07:12:38.114 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 378: collecting document frequencies
2019-03-06 07:12:38.122 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}
2019-03-06 07:12:38.127 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 384: PROGRESS: processing document #0
2019-03-06 07:12:38.132 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 396: calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [61]:
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

2019-03-06 07:12:47.193 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 420: using serial LSI version on this node
2019-03-06 07:12:47.202 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 466: updating model with new documents
2019-03-06 07:12:47.207 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}
2019-03-06 07:12:47.211 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 492: preparing a new chunk of documents
2019-03-06 07:12:47.214 DEBUG in 'gensim.models.lsimodel'['lsimodel'] at line 496: converting corpus to csc format
2019-03-06 07:12:47.218 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 912: using 100 extra samples and 2 power iterations
2019-03-06 07:12:47.222 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 920: 1st phase: constructing (12, 400) action matrix
2019-03-06 07:12:47.229 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 935: orthonormal

In [62]:
another_tfidf_corpus = corpus_tfidf
model.add_documents(another_tfidf_corpus) # now LSI has been trained on tfidf_corpus + another_tfidf_corpus lsi_vec = model[tfidf_vec] # convert some new document into the LSI space, without affecting the model

more_documents = corpus_tfidf
model.add_documents(more_documents) # tfidf_corpus + another_tfidf_corpus + more_documents lsi_vec = model[tfidf_vec]

2019-03-06 07:12:57.291 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 466: updating model with new documents
2019-03-06 07:12:57.300 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}
2019-03-06 07:12:57.304 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 492: preparing a new chunk of documents
2019-03-06 07:12:57.308 DEBUG in 'gensim.models.lsimodel'['lsimodel'] at line 496: converting corpus to csc format
2019-03-06 07:12:57.312 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 912: using 100 extra samples and 2 power iterations
2019-03-06 07:12:57.315 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 920: 1st phase: constructing (12, 400) action matrix
2019-03-06 07:12:57.318 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 935: orthonormalizing (12, 400) action matrix
2019-03-06 07:12:57.322 DEBUG in 'gensim.matutils'['matutils'] at line 1176: computing QR 

In [63]:
model = models.RpModel(corpus_tfidf, num_topics=500)

2019-03-06 07:13:09.592 INFO in 'gensim.models.rpmodel'['rpmodel'] at line 84: no word id mapping provided; initializing from corpus, assuming identity
2019-03-06 07:13:09.598 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}
2019-03-06 07:13:09.603 INFO in 'gensim.models.rpmodel'['rpmodel'] at line 93: constructing (500, 12) random matrix


In [64]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

2019-03-06 07:13:13.660 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric alpha at 0.01
2019-03-06 07:13:13.666 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric eta at 0.01
2019-03-06 07:13:13.672 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 481: using serial LDA version on this node
2019-03-06 07:13:13.724 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (single-pass) LDA training, 100 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2019-03-06 07:13:13.727 WARNING in 'gensim.models.ldamodel'['ldamodel'] at line 924: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-03-06 07:13:13.731 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\U

In [70]:
model = models.HdpModel(corpus, id2word=dictionary)

2019-03-06 07:13:57.795 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\deerwester.mm'}
2019-03-06 07:13:57.819 INFO in 'gensim.models.hdpmodel'['hdpmodel'] at line 1062: (0, '0.366*computer + 0.142*user + 0.120*human + 0.104*survey + 0.064*trees + 0.056*system + 0.052*interface + 0.037*eps + 0.023*graph + 0.023*minors')
2019-03-06 07:13:57.822 INFO in 'gensim.models.hdpmodel'['hdpmodel'] at line 1062: (1, '0.196*trees + 0.136*system + 0.134*graph + 0.118*time + 0.106*user + 0.097*human + 0.081*interface + 0.073*survey + 0.030*computer + 0.016*eps')
2019-03-06 07:13:57.826 INFO in 'gensim.models.hdpmodel'['hdpmodel'] at line 1062: (2, '0.240*graph + 0.192*response + 0.112*user + 0.107*time + 0.091*computer + 0.074*interface + 0.069*human + 0.044*trees + 0.035*system + 0.015*survey')
2019-03-06 07:13:57.829 INFO in 'gensim.models.hdpmodel'['hdpmodel'] at line 1062: (3, '0.222*minors + 0.168*trees

In [71]:
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
model = LsiModel(common_corpus, id2word=common_dictionary)
vectorized_corpus = model[common_corpus]  # vectorize input copus in BoW format

2019-03-06 07:14:09.768 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 420: using serial LSI version on this node
2019-03-06 07:14:09.776 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 466: updating model with new documents
2019-03-06 07:14:09.779 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 492: preparing a new chunk of documents
2019-03-06 07:14:09.783 DEBUG in 'gensim.models.lsimodel'['lsimodel'] at line 496: converting corpus to csc format
2019-03-06 07:14:09.787 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 912: using 100 extra samples and 2 power iterations
2019-03-06 07:14:09.790 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 920: 1st phase: constructing (12, 300) action matrix
2019-03-06 07:14:09.793 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 935: orthonormalizing (12, 300) action matrix
2019-03-06 07:14:09.798 DEBUG in 'gensim.matutils'['matutils'] at line 1176: computing QR of (12, 300) dense matrix
2019-03-06 07:14:09.802 DEBUG in '

In [73]:
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
model = LsiModel(common_corpus[:3], id2word=common_dictionary)  # train model
vector = model[common_corpus[4]]  # apply model to BoW document
model.add_documents(common_corpus[4:])  # update model with new documents
tmp_fname = get_tmpfile("lsi.model")
model.save(tmp_fname)  # save model
loaded_model = LsiModel.load(tmp_fname)  # load model

2019-03-06 07:14:42.379 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 420: using serial LSI version on this node
2019-03-06 07:14:42.389 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 466: updating model with new documents
2019-03-06 07:14:42.396 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 492: preparing a new chunk of documents
2019-03-06 07:14:42.400 DEBUG in 'gensim.models.lsimodel'['lsimodel'] at line 496: converting corpus to csc format
2019-03-06 07:14:42.403 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 912: using 100 extra samples and 2 power iterations
2019-03-06 07:14:42.406 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 920: 1st phase: constructing (12, 300) action matrix
2019-03-06 07:14:42.410 INFO in 'gensim.models.lsimodel'['lsimodel'] at line 935: orthonormalizing (12, 300) action matrix
2019-03-06 07:14:42.413 DEBUG in 'gensim.matutils'['matutils'] at line 1176: computing QR of (12, 300) dense matrix
2019-03-06 07:14:42.416 DEBUG in '

In [74]:
from gensim.models import RpModel
from gensim.corpora import Dictionary
from gensim.test.utils import common_texts, temporary_file
dictionary = Dictionary(common_texts)  # fit dictionary
corpus = [dictionary.doc2bow(text) for text in common_texts]  # convert texts to BoW format
model = RpModel(corpus, id2word=dictionary)  # fit model
result = model[corpus[3]]  # apply model to document, result is vector in BoW format
with temporary_file("model_file") as fname:
    model.save(fname)  # save model to file
    loaded_model = RpModel.load(fname)  # load model

2019-03-06 07:14:56.19 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 202: adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 07:14:56.22 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 209: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2019-03-06 07:14:56.26 INFO in 'gensim.models.rpmodel'['rpmodel'] at line 93: constructing (300, 12) random matrix
2019-03-06 07:14:56.124 INFO in 'gensim.utils'['utils'] at line 542: saving RpModel object under C:\Users\Home\AppData\Local\Temp\tmpzeanjqzd\model_file, separately None
2019-03-06 07:14:56.133 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\AppData\\Local\\Temp\\tmpzeanjqzd\\model_file'}
2019-03-06 07:14:56.139 INFO in 'gensim.utils'['utils'] at line 556: saved C:\Users\Home\AppData\Local\Temp\tmpzeanjqzd\model_file
2019-03-06 07:14:56.149 INF

In [75]:

from gensim.models import RpModel
from gensim.corpora import Dictionary
from gensim.test.utils import common_texts
dictionary = Dictionary(common_texts)  # fit dictionary
corpus = [dictionary.doc2bow(text) for text in common_texts]  # convert texts to BoW format
model = RpModel(corpus, id2word=dictionary)  # fit model
# apply model to document, result is vector in BoW format, i.e. [(1, 0.3), ... ]
result = model[corpus[0]]

2019-03-06 07:15:03.602 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 202: adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 07:15:03.609 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 209: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2019-03-06 07:15:03.612 INFO in 'gensim.models.rpmodel'['rpmodel'] at line 93: constructing (300, 12) random matrix


In [76]:
#Train an LDA model using a Gensim corpus

from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)

2019-03-06 07:15:09.273 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 202: adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 07:15:09.280 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 209: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2019-03-06 07:15:09.283 WARNING in 'gensim.models.ldamodel'['ldamodel'] at line 432: no word id mapping provided; initializing from corpus, assuming identity
2019-03-06 07:15:09.285 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric alpha at 0.1
2019-03-06 07:15:09.287 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric eta at 0.1
2019-03-06 07:15:09.289 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 481: using serial LDA version on this node
2019-03-06 07:15:09.307 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (single-pass) LDA training, 10 topic

In [77]:
#Save a model to disk, or reload a pre-trained model
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("model")
lda.save(temp_file)
# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

2019-03-06 07:15:19.207 INFO in 'gensim.utils'['utils'] at line 542: saving LdaState object under C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\gensim\test\test_data\model.state, separately None
2019-03-06 07:15:19.210 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\model.state'}
2019-03-06 07:15:19.231 INFO in 'gensim.utils'['utils'] at line 556: saved C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\gensim\test\test_data\model.state
2019-03-06 07:15:19.235 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\model.id2word'}
2019-03-06 07:15:19.239 INFO in 'gensim.utils'['utils'] at line 542: saving LdaModel object under C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\gensim\t

In [78]:
#Query, the model using new, unseen documents
# Create a new corpus, made of previously unseen documents.
other_texts = [
    ['computer', 'time', 'graph'],
    ['survey', 'response', 'eps'],
    ['human', 'system', 'computer']
]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]
unseen_doc = other_corpus[0]
vector = lda[unseen_doc]  # get topic probability distribution for a document

In [79]:
#Update the model by incrementally training on the new corpus
lda.update(other_corpus)
vector = lda[unseen_doc]

2019-03-06 07:15:32.891 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 3 documents, updating model once every 3 documents, evaluating perplexity every 3 documents, iterating 50x with a convergence threshold of 0.001000
2019-03-06 07:15:32.899 WARNING in 'gensim.models.ldamodel'['ldamodel'] at line 924: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-03-06 07:15:32.905 DEBUG in 'gensim.models.ldamodel'['ldamodel'] at line 1073: bound: at document #0
2019-03-06 07:15:32.911 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 814: -7.827 per-word bound, 227.1 perplexity estimate based on a held-out corpus of 3 documents with 9 words
2019-03-06 07:15:32.914 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 968: PROGRESS: pass 0, at document #3/3
2019-03-06 07:15:32.917 DEBUG in 'gensim.models.ldamodel'['ldamo

In [80]:
#A lot of parameters can be tuned to optimize training for your specific type
lda = LdaModel(common_corpus, num_topics=50, alpha='auto', eval_every=5)  # learn asymmetric alpha from data

2019-03-06 07:15:41.99 WARNING in 'gensim.models.ldamodel'['ldamodel'] at line 432: no word id mapping provided; initializing from corpus, assuming identity
2019-03-06 07:15:41.108 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 570: using autotuned alpha, starting with [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
2019-03-06 07:15:41.114 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric eta at 0.02
2019-03-06 07:15:41.117 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 481: using serial LDA version on this node
2019-03-06 07:15:41.121 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (single-pass) LDA training, 50 topics, 1 passes over the supplied corpus of 9 documents, updatin

In [81]:
#Initialize a model using a Gensim corpus
from gensim.test.utils import common_corpus
lda = LdaModel(common_corpus, num_topics=10)

2019-03-06 07:15:57.890 WARNING in 'gensim.models.ldamodel'['ldamodel'] at line 432: no word id mapping provided; initializing from corpus, assuming identity
2019-03-06 07:15:57.899 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric alpha at 0.1
2019-03-06 07:15:57.904 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric eta at 0.1
2019-03-06 07:15:57.908 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 481: using serial LDA version on this node
2019-03-06 07:15:57.911 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2019-03-06 07:15:57.915 WARNING in 'gensim.models.ldamodel'['ldamodel'] at line 924: too few updates, training might not converge; consider increasing the number of passes or iterat

In [82]:
#You can then infer topic distributions on new, unseen documents.
doc_bow = [(1, 0.3), (2, 0.1), (0, 0.09)]
doc_lda = lda[doc_bow]

In [83]:
#The model can be updated (trained) with new documents.
# In practice (corpus =/= initial training corpus), but we use the same here for simplicity.
other_corpus = common_corpus
lda.update(other_corpus)

2019-03-06 07:16:08.17 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2019-03-06 07:16:08.21 WARNING in 'gensim.models.ldamodel'['ldamodel'] at line 924: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-03-06 07:16:08.24 DEBUG in 'gensim.models.ldamodel'['ldamodel'] at line 1073: bound: at document #0
2019-03-06 07:16:08.31 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 814: -4.152 per-word bound, 17.8 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2019-03-06 07:16:08.34 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 968: PROGRESS: pass 0, at document #9/9
2019-03-06 07:16:08.38 DEBUG in 'gensim.models.ldamodel'['ldamodel'] 

In [84]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.test.utils import datapath
m1 = LdaMulticore.load(datapath("lda_3_0_1_model"))
m2 = LdaMulticore.load(datapath("ldamodel_python_3_5"))
mdiff, annotation = m1.diff(m2)
topic_diff = mdiff  # get matrix with difference for each topic pair from `m1` and `m2

2019-03-06 07:16:17.311 INFO in 'gensim.utils'['utils'] at line 422: loading LdaMulticore object from C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\gensim\test\test_data\lda_3_0_1_model
2019-03-06 07:16:17.315 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\lda_3_0_1_model'}
2019-03-06 07:16:17.430 INFO in 'gensim.utils'['utils'] at line 461: loading expElogbeta from C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\gensim\test\test_data\lda_3_0_1_model.expElogbeta.npy with mmap=None
2019-03-06 07:16:17.451 INFO in 'gensim.utils'['utils'] at line 494: setting ignored attribute id2word to None
2019-03-06 07:16:17.457 INFO in 'gensim.utils'['utils'] at line 494: setting ignored attribute state to None
2019-03-06 07:16:17.461 INFO in 'gensim.utils'['utils'] at line 494: setting ignored attribute dispatcher to None
2019-03-06 07

In [85]:
#Large arrays can be memmap’ed back as read-only (shared memory) by setting mmap=’r’

from gensim.test.utils import datapath
fname = datapath("lda_3_0_1_model")
lda = LdaModel.load(fname, mmap='r')

2019-03-06 07:16:28.547 INFO in 'gensim.utils'['utils'] at line 422: loading LdaModel object from C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\gensim\test\test_data\lda_3_0_1_model
2019-03-06 07:16:28.551 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\lda_3_0_1_model'}
2019-03-06 07:16:28.558 INFO in 'gensim.utils'['utils'] at line 461: loading expElogbeta from C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\gensim\test\test_data\lda_3_0_1_model.expElogbeta.npy with mmap=r
2019-03-06 07:16:28.577 INFO in 'gensim.utils'['utils'] at line 494: setting ignored attribute id2word to None
2019-03-06 07:16:28.581 INFO in 'gensim.utils'['utils'] at line 494: setting ignored attribute state to None
2019-03-06 07:16:28.584 INFO in 'gensim.utils'['utils'] at line 494: setting ignored attribute dispatcher to None
2019-03-06 07:16:28.

In [86]:
corpus = gensim.corpora.MalletCorpus('android.mallet')

2019-03-06 07:16:40.721 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'android.mallet.index'}
2019-03-06 07:16:40.730 INFO in 'gensim.corpora.lowcorpus'['lowcorpus'] at line 97: loading corpus from android.mallet
2019-03-06 07:16:40.735 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'android.mallet'}
2019-03-06 07:16:40.983 INFO in 'gensim.corpora.lowcorpus'['lowcorpus'] at line 105: extracting vocabulary from the corpus
2019-03-06 07:16:40.985 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'android.mallet'}
2019-03-06 07:16:41.575 INFO in 'gensim.corpora.lowcorpus'['lowcorpus'] at line 121: loaded corpus with 37622 documents and 16405 terms from android.mallet


In [87]:
next(iter(corpus))

2019-03-06 07:16:42.962 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'android.mallet'}


[(6936, 1), (15314, 1), (300, 1), (10981, 1)]

In [88]:
for word_id, freq in next(iter(corpus)):
    print(corpus.id2word[word_id], freq)

2019-03-06 07:16:51.832 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'android.mallet'}


incorrect 1
url 1
address 1
project 1


In [89]:
model = gensim.models.LdaModel(corpus, id2word=corpus.id2word, alpha='auto', num_topics=25)
model.save('android.lda')
#model = gensim.models.LdaModel.load('android.lda')

2019-03-06 07:16:52.835 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 570: using autotuned alpha, starting with [0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]
2019-03-06 07:16:52.846 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric eta at 0.04
2019-03-06 07:16:52.853 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 481: using serial LDA version on this node
2019-03-06 07:16:52.914 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (single-pass) LDA training, 25 topics, 1 passes over the supplied corpus of 37622 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-03-06 07:16:52.917 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'android.mallet'}
2019-03-06 07:16:52.964 IN

2019-03-06 07:16:58.906 DEBUG in 'gensim.models.ldamodel'['ldamodel'] at line 700: 1999/2000 documents converged within 50 iterations
2019-03-06 07:16:58.935 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 759: optimized alpha [0.043068044, 0.040774483, 0.04073559, 0.042248018, 0.039673667, 0.041417763, 0.039941516, 0.04052188, 0.041213647, 0.03980672, 0.040612575, 0.040907774, 0.040277302, 0.041145094, 0.04091007, 0.041131712, 0.03836616, 0.038600326, 0.041759185, 0.039450943, 0.039997943, 0.04087076, 0.038579643, 0.038479313, 0.03920821]
2019-03-06 07:16:58.938 DEBUG in 'gensim.models.ldamodel'['ldamodel'] at line 1027: updating topics
2019-03-06 07:16:58.950 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 230: merging changes from 2000 documents into a model of 37622 documents
2019-03-06 07:16:59.21 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #16 (0.038): 0.109*"email" + 0.057*"account" + 0.043*"send" + 0.040*"app" + 0.036*"emails" + 0.029*"native" + 0

2019-03-06 07:17:03.764 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #22 (0.040): 0.068*"camera" + 0.067*"network" + 0.062*"location" + 0.032*"gps" + 0.030*"don" + 0.030*"work" + 0.025*"reply" + 0.024*"login" + 0.023*"storage" + 0.021*"wifi"
2019-03-06 07:17:03.770 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #24 (0.041): 0.105*"voice" + 0.066*"view" + 0.062*"lock" + 0.049*"google" + 0.043*"screen" + 0.035*"reboot" + 0.034*"map" + 0.032*"options" + 0.023*"day" + 0.020*"calling"
2019-03-06 07:17:03.776 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #14 (0.047): 0.064*"exception" + 0.056*"wifi" + 0.048*"java" + 0.048*"navigation" + 0.042*"bar" + 0.037*"notification" + 0.030*"setting" + 0.028*"keypad" + 0.023*"window" + 0.021*"status"
2019-03-06 07:17:03.786 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #3 (0.048): 0.133*"close" + 0.126*"force" + 0.041*"browser" + 0.033*"battery" + 0.032*"android" + 0.027*"input"

2019-03-06 07:17:07.947 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #3 (0.055): 0.208*"force" + 0.207*"close" + 0.031*"battery" + 0.027*"input" + 0.020*"browser" + 0.019*"photo" + 0.018*"icon" + 0.018*"android" + 0.018*"touch" + 0.016*"background"
2019-03-06 07:17:07.959 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #5 (0.060): 0.240*"google" + 0.120*"maps" + 0.070*"android" + 0.066*"error" + 0.034*"incorrect" + 0.033*"displaying" + 0.032*"widget" + 0.029*"code" + 0.020*"crashed" + 0.019*"ndk"
2019-03-06 07:17:07.967 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1037: topic diff=0.043868, rho=0.229416
2019-03-06 07:17:07.977 INFO in 'gensim.utils'['utils'] at line 542: saving LdaState object under android.lda.state, separately None
2019-03-06 07:17:07.983 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'wb', 'uri': 'android.lda.state'}
2019-03-06 07:17:08.9 INFO in 'gensim.utils'['utils'] at line 556:

In [90]:
model.show_topics(num_topics=5, formatted=True, num_words = 3)

[(22, '0.068*"location" + 0.068*"camera" + 0.062*"network"'),
 (16, '0.140*"email" + 0.132*"account" + 0.083*"app"'),
 (14, '0.058*"wifi" + 0.055*"java" + 0.049*"navigation"'),
 (3, '0.208*"force" + 0.207*"close" + 0.031*"battery"'),
 (5, '0.240*"google" + 0.120*"maps" + 0.070*"android"')]

In [91]:
model.show_topic(topicid = 6, topn = 10)

[('update', 0.10460339),
 ('settings', 0.0961843),
 ('store', 0.079363555),
 ('call', 0.06707138),
 ('displays', 0.039056607),
 ('android', 0.032985814),
 ('tab', 0.03022936),
 ('developer', 0.025293238),
 ('applications', 0.02174573),
 ('process', 0.02065673)]

In [93]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

2019-03-06 07:18:35.662 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 202: adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 07:18:35.672 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 209: built Dictionary(32 unique tokens: ['brocolli', 'brother', 'eat', 'good', 'like']...) from 5 documents (total 46 corpus positions)
2019-03-06 07:18:35.686 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric alpha at 0.5
2019-03-06 07:18:35.700 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 557: using symmetric eta at 0.5
2019-03-06 07:18:35.706 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 481: using serial LDA version on this node
2019-03-06 07:18:35.711 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 919: running online (multi-pass) LDA training, 2 topics, 20 passes over the supplied corpus of 5 documents, updating model once every 5 documents, evaluating perplexity every 5 documents, iterating 50x with a convergence thr

2019-03-06 07:18:36.312 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1159: topic #1 (0.500): 0.060*"health" + 0.058*"pressur" + 0.058*"drive" + 0.057*"increas" + 0.057*"may" + 0.057*"expert" + 0.057*"caus" + 0.057*"suggest" + 0.057*"blood" + 0.057*"tension"
2019-03-06 07:18:36.315 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 1037: topic diff=0.001899, rho=0.288675
2019-03-06 07:18:36.319 DEBUG in 'gensim.models.ldamodel'['ldamodel'] at line 1073: bound: at document #0
2019-03-06 07:18:36.325 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 814: -3.924 per-word bound, 15.2 perplexity estimate based on a held-out corpus of 5 documents with 46 words
2019-03-06 07:18:36.328 INFO in 'gensim.models.ldamodel'['ldamodel'] at line 968: PROGRESS: pass 11, at document #5/5
2019-03-06 07:18:36.331 DEBUG in 'gensim.models.ldamodel'['ldamodel'] at line 638: performing inference on a chunk of 5 documents
2019-03-06 07:18:36.336 DEBUG in 'gensim.models.ldamodel'['ldamodel'] at line

In [94]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
dataset = api.load("text8")
dct = Dictionary(dataset)  # fit dictionary
corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format
model = TfidfModel(corpus)  # fit model
vector = model[corpus[0]]
print(vector)

2019-03-06 07:19:04.505 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\gensim-data\\text8\\text8.gz'}
2019-03-06 07:19:04.574 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 202: adding document #0 to Dictionary(0 unique tokens: [])
2019-03-06 07:19:22.757 INFO in 'gensim.corpora.dictionary'['dictionary'] at line 209: built Dictionary(253854 unique tokens: ['a', 'abacus', 'abilities', 'ability', 'able']...) from 1701 documents (total 17005207 corpus positions)
2019-03-06 07:19:22.763 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\gensim-data\\text8\\text8.gz'}
2019-03-06 07:19:37.951 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 378: collecting document frequencies
2019-03-06 07:19:37.954 INFO in 'gensim.models.tfidfmodel'['tfidfmodel'] at line 384: PROGRESS: processing document #0
2019-03-06 07:19:39.383 INFO in 'gensim.mode

[(1, 0.006704047545684609), (2, 0.0030255603220721273), (3, 0.003156168449586299), (4, 0.0036673470201144674), (5, 0.004575122435127926), (6, 0.0028052608258295926), (7, 0.004064820137019515), (8, 0.00014963587508918375), (9, 0.0007492665180478759), (10, 0.004142807322609117), (11, 0.004149816941645728), (12, 0.0077498817493309525), (13, 0.00656024165742503), (14, 0.003891486499758776), (15, 0.005476877392392166), (16, 0.0018233938817994433), (17, 0.0032209070754237084), (18, 0.0017737283389229173), (19, 0.0023373507198140124), (20, 0.003725514968930464), (21, 0.00590342512385848), (22, 0.003072401062545206), (23, 0.0006668171096292247), (24, 0.0017594266221832493), (25, 0.004202080158963513), (26, 0.002967397324595724), (27, 0.004709756138185673), (28, 0.0014819657487289912), (29, 0.0031562459553171694), (30, 0.0031999829254611097), (31, 0.001215574949729317), (32, 0.003843126241898761), (33, 0.006499414537896336), (34, 0.004546489373863172), (35, 0.003855345600331923), (36, 0.0046313

In [95]:
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser
sentences = Text8Corpus(datapath('testcorpus.txt'))
phrases = Phrases(sentences, min_count=1, threshold=1)  # train model
print(phrases[[u'trees', u'graph', u'minors']])  # apply model to sentence
phrases.add_vocab([["hello", "world"], ["meow"]])  # update model with new sentences
bigram = Phraser(phrases)  # construct faster model (this is only an wrapper)
print(bigram[[u'trees', u'graph', u'minors']])  # apply model to sentence
for sent in bigram[sentences]:  # apply model to text corpus
     pass

2019-03-06 07:42:22.85 INFO in 'gensim.models.phrases'['phrases'] at line 475: collecting all words and their counts
2019-03-06 07:42:22.91 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\testcorpus.txt'}
2019-03-06 07:42:22.119 INFO in 'gensim.models.phrases'['phrases'] at line 482: PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-06 07:42:22.145 INFO in 'gensim.models.phrases'['phrases'] at line 505: collected 37 word types from a corpus of 29 words (unigram + bigrams) and 1 sentences
2019-03-06 07:42:22.150 INFO in 'gensim.models.phrases'['phrases'] at line 558: using 37 counts as vocab in Phrases<0 vocab, min_count=1, threshold=1, max_vocab_size=40000000>


['trees_graph', 'minors']


2019-03-06 07:42:22.156 INFO in 'gensim.models.phrases'['phrases'] at line 475: collecting all words and their counts
2019-03-06 07:42:22.160 INFO in 'gensim.models.phrases'['phrases'] at line 482: PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-06 07:42:22.163 INFO in 'gensim.models.phrases'['phrases'] at line 505: collected 4 word types from a corpus of 3 words (unigram + bigrams) and 2 sentences
2019-03-06 07:42:22.166 INFO in 'gensim.models.phrases'['phrases'] at line 548: merging 4 counts into Phrases<37 vocab, min_count=1, threshold=1, max_vocab_size=40000000>
2019-03-06 07:42:22.170 INFO in 'gensim.models.phrases'['phrases'] at line 555: merged Phrases<41 vocab, min_count=1, threshold=1, max_vocab_size=40000000>
2019-03-06 07:42:22.174 INFO in 'gensim.models.phrases'['phrases'] at line 816: source_vocab length 41
2019-03-06 07:42:22.178 INFO in 'gensim.models.phrases'['phrases'] at line 825: Phraser built with 3 phrasegrams


['trees_graph', 'minors']


2019-03-06 07:42:22.184 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\testcorpus.txt'}
2019-03-06 07:42:22.187 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\testcorpus.txt'}


In [96]:
phrases = Phrases(sentences)  # train model
assert len(phrases.vocab) == 37
more_sentences = [
     [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there'],
     [u'machine', u'learning', u'can', u'be', u'new', u'york', u'sometimes']
                 ]
phrases.add_vocab(more_sentences)  # add new sentences to model
assert len(phrases.vocab) == 60

2019-03-06 07:42:31.561 INFO in 'gensim.models.phrases'['phrases'] at line 475: collecting all words and their counts
2019-03-06 07:42:31.569 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\testcorpus.txt'}
2019-03-06 07:42:31.575 INFO in 'gensim.models.phrases'['phrases'] at line 482: PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-06 07:42:31.581 INFO in 'gensim.models.phrases'['phrases'] at line 505: collected 37 word types from a corpus of 29 words (unigram + bigrams) and 1 sentences
2019-03-06 07:42:31.587 INFO in 'gensim.models.phrases'['phrases'] at line 558: using 37 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2019-03-06 07:42:31.591 INFO in 'gensim.models.phrases'['phrases'] at line 475: collecting all words and their counts
2019-03-06 07:42:31.594 INFO in 'gensim.model

In [97]:
phrases = Phrases(sentences, min_count=1, threshold=0.1)
for phrase, score in phrases.export_phrases(sentences):
     pass

2019-03-06 07:42:40.169 INFO in 'gensim.models.phrases'['phrases'] at line 475: collecting all words and their counts
2019-03-06 07:42:40.179 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\testcorpus.txt'}
2019-03-06 07:42:40.183 INFO in 'gensim.models.phrases'['phrases'] at line 482: PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-06 07:42:40.186 INFO in 'gensim.models.phrases'['phrases'] at line 505: collected 37 word types from a corpus of 29 words (unigram + bigrams) and 1 sentences
2019-03-06 07:42:40.189 INFO in 'gensim.models.phrases'['phrases'] at line 558: using 37 counts as vocab in Phrases<0 vocab, min_count=1, threshold=0.1, max_vocab_size=40000000>
2019-03-06 07:42:40.194 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tenso

In [98]:
pruned_words, counters, total_words = Phrases.learn_vocab(sentences, 100)
print(pruned_words, total_words)
print(counters['computer'])
print(counters['response_time'])

2019-03-06 07:42:41.618 INFO in 'gensim.models.phrases'['phrases'] at line 475: collecting all words and their counts
2019-03-06 07:42:41.626 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\Anaconda2\\envs\\tensorflow\\lib\\site-packages\\gensim\\test\\test_data\\testcorpus.txt'}
2019-03-06 07:42:41.633 INFO in 'gensim.models.phrases'['phrases'] at line 482: PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-06 07:42:41.638 INFO in 'gensim.models.phrases'['phrases'] at line 505: collected 37 word types from a corpus of 29 words (unigram + bigrams) and 1 sentences


1 29
0
0


In [1]:
import nltk
import csv
import string
from gensim.models import Phrases
from gensim.models import Word2Vec
from nltk.corpus import stopwords

sentences = []
bigram = Phrases()
rows = [] 

with open("sentences.csv", "r") as sentencesfile:
    reader = csv.reader(sentencesfile, delimiter = ",")
    next(reader)
    for row in reader: 
        rows.append(row) 
    
    
    for row in rows:
        sentence = [word 
                    for word in nltk.word_tokenize(row[4].lower())
                    if word not in string.punctuation]
        sentences.append(sentence)
        bigram.add_vocab([sentence])


In [2]:
list(bigram[sentences])[:5]

[['pilot'],
 ['scene', 'one'],
 ['title', 'the', 'year', '2030'],
 ['narrator_kids',
  'i',
  "'m",
  'going',
  'to',
  'tell',
  'you',
  'an',
  'incredible',
  'story',
  'the',
  'story',
  'of',
  'how',
  'i',
  'met_your',
  'mother'],
 ['son', 'are', 'we', 'being', 'punished', 'for', 'something']]

In [3]:
#populate a Counter with our phrases and their counts and find out the more common phrases

from collections import Counter
i = 0
bigram_counter = Counter()
for key1 in list(bigram.vocab):
    #key1 = key
    #print(type(key1))
    
  
    if key1 not in stopwords.words("english"):
        #print(type(key))
        byte1 = "_".encode()
        if len(key1.split(byte1)) > 1:
            #print("ok")
            bigram_counter[key1] += bigram.vocab[key1]
            print(bigram.vocab[key1])
            i=i+1
        if i > 10: 
            break;

#print(i)
for key10, counts in bigram_counter.most_common(20):
    #print(key10)
    print("{}       {}".format(key10.decode(), counts))


2
14
57
21
91
38
4607
232
827
189
247
i_'m       4607
going_to       827
tell_you       247
'm_going       232
to_tell       189
narrator_kids       91
the_year       57
kids_i       38
year_2030       21
title_the       14
scene_one       2


In [4]:
from collections import Counter

model = Word2Vec(bigram[sentences], size=100)

i = 0
bigram_counter = Counter()
for key1 in list(model.wv.vocab):
#for key1 in model.wv.vocab.keys():    
    #key = key1
    #print(type(key1))
    
    if key1 not in stopwords.words("english"):
        #print(type(key))
        #byte1 = "_".encode()
        if len(key1.split("_")) > 1:
            #print("ok")
            bigram_counter[key1] += model.wv.vocab[key1].count
            #print(model.wv.vocab[key].count)
            #i=i+1
        #if i > 10: 
            #break;

#print(i)
for key10, counts in bigram_counter.most_common(20):
    #print(key10)
    print("{}       {}".format(key10, counts))


do_n't       2459
gon_na       1576
ca_n't       1034
did_n't       719
come_on       587
end_of       460
'm_sorry       453
my_god       442
kind_of       398
from_2030       394
all_right       354
they_'re       351
does_n't       347
end_flashback       328
of_course       322
right_now       314
've_been       303
new_york       300
a_lot       297
look_at       265


In [5]:
model.most_similar(['marshall', 'lily'], ['ted'], topn=10)

C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('robin', 0.859014093875885),
 ('barney', 0.8380885720252991),
 ('victoria', 0.81501305103302),
 ('zoey', 0.80119389295578),
 ('james', 0.7664534449577332),
 ('claudia', 0.7588191032409668),
 ('nora', 0.7587673664093018),
 ('don', 0.7442715167999268),
 ('brad', 0.7167814373970032),
 ('totally', 0.7081925272941589)]

In [6]:
model.similarity("ted", "robin")

C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.92367536

In [7]:
model.similarity("barney", "robin")

C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.96717703

In [8]:
model.most_similar(positive=['getting_married'])

C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('killing_me', 0.9587925672531128),
 ('having_fun', 0.9442540407180786),
 ('freaking_out', 0.9415212869644165),
 ('delightful', 0.9413829445838928),
 ('an_idiot', 0.9392070770263672),
 ('having_trouble', 0.9387280344963074),
 ('lucky', 0.9348148703575134),
 ('so_glad', 0.9323379993438721),
 ('scared', 0.9287667274475098),
 ('breaking_up', 0.9279994368553162)]

In [9]:
model.most_similar(positive=['my_dad'])

C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('your_mother', 0.9219773411750793),
 ('quinn', 0.917591392993927),
 ('our_relationship', 0.9173168540000916),
 ('mistakes', 0.9126814007759094),
 ('shy', 0.9087252020835876),
 ('devastating', 0.9079723358154297),
 ('his_name', 0.907180666923523),
 ('suspicious', 0.9052258133888245),
 ('polite', 0.9039556384086609),
 ('pissed', 0.9008902311325073)]

In [10]:
import gensim

In [11]:
gensim.__version__

'3.7.0'

In [12]:
cnt = Counter()
for word in ['red', 'red', 'yellow']:
    cnt[word] += 1
print(cnt)

import re
words = re.findall(r'\w+', open('hamlet.txt').read().lower())
Counter(words).most_common(10)

Counter({'red': 2, 'yellow': 1})


[('the', 993),
 ('and', 863),
 ('to', 685),
 ('of', 610),
 ('i', 574),
 ('you', 527),
 ('a', 511),
 ('my', 502),
 ('it', 419),
 ('in', 400)]

In [13]:
c = Counter()
c['sausage'] = 0                        # counter entry with a zero count
del c['sausage']        

In [14]:
sum(c.values())                 # total of all counts
c.clear()                       # reset all counts
list(c)                         # list unique elements
set(c)                          # convert to a set
dict(c)                         # convert to a regular dictionary
c.items()                       # convert to a list of (elem, cnt) pairs
list_of_pairs = {'a': 3, 'b': 0, 'c': -3, 'd': -6}
Counter(dict(list_of_pairs))    # convert from a list of (elem, cnt) pairs
n = 10
c.most_common()[:-n-1:-1]       # n least common elements
c += Counter()                  # remove zero and negative counts

In [15]:
c = Counter(a=3, b=1)
d = Counter(a=1, b=2)
print(c+d)
print(c-d)
print(c & d)                       # intersection:  min(c[x], d[x])
print(c | d)                       # union:  max(c[x], d[x])

Counter({'a': 4, 'b': 3})
Counter({'a': 2})
Counter({'a': 1, 'b': 1})
Counter({'a': 3, 'b': 2})


In [16]:
from collections import deque
d = deque('ghi')                 # make a new deque with three items
for elem in d:                   # iterate over the deque's elements
    print(elem.upper())

G
H
I


In [17]:
d.append('j')                    # add a new entry to the right side
print(d)
d.appendleft('f')                # add a new entry to the left side
print(d)                                # show the representation of the deque

deque(['g', 'h', 'i', 'j'])
deque(['f', 'g', 'h', 'i', 'j'])


In [18]:
d.pop()                          # return and remove the rightmost item
print(d)
d.popleft()                      # return and remove the leftmost item
print(d)

deque(['f', 'g', 'h', 'i'])
deque(['g', 'h', 'i'])


In [19]:
list(d)                          # list the contents of the deque
print(d[0])                             # peek at leftmost item
print(d[-1])                            # peek at rightmost item

g
i


In [20]:
list(reversed(d))                # list the contents of a deque in reverse
print('h' in d)                         # search the deque

True


In [21]:
d.extend('jkl')                  # add multiple elements at once
print(d)

deque(['g', 'h', 'i', 'j', 'k', 'l'])


In [22]:
d.rotate(1)                      # right rotation
print(d)
print(d.rotate(-1))                     # left rotation
print(d)

deque(['l', 'g', 'h', 'i', 'j', 'k'])
None
deque(['g', 'h', 'i', 'j', 'k', 'l'])


In [23]:
print(deque(reversed(d)))               # make a new deque in reverse order
print(d.clear())                        # empty the deque
#print(d.pop())                          # cannot pop from an empty deque
d.extendleft('abc')              # extendleft() reverses the input order
print(d)

deque(['l', 'k', 'j', 'i', 'h', 'g'])
None
deque(['c', 'b', 'a'])


In [24]:
from collections import defaultdict
s = [('yellow', 1), ('red', 3), ('yellow', 3), ('red', 6), ('red', 1)]
d = defaultdict(list)
for k, v in s:
    d[k].append(v)
print(d.items())

dict_items([('yellow', [1, 3]), ('red', [3, 6, 1])])


In [25]:
#This technique is simpler and faster than an equivalent technique using defaultdict.setdefault()
d = {}
for k, v in s:
    d.setdefault(k, []).append(v)
print(d.items())

dict_items([('yellow', [1, 3]), ('red', [3, 6, 1])])


In [26]:
s = 'mississippi'
d = defaultdict(int)
for k in s:
    d[k] += 1
print(d.items())

dict_items([('m', 1), ('i', 4), ('s', 4), ('p', 2)])


In [27]:
import itertools

def constant_factory(value):
    return next(itertools.repeat(value))
d = defaultdict(constant_factory(lambda:'<missing>'))
d.update(name='John', action='ran')
print('%(name)s %(action)s to %(object)s' % d)

John ran to <missing>


In [28]:
#Setting the default_factory to set makes the defaultdict useful for building a dictionary of sets

s = [('red', 1), ('orange', 2), ('red', 3), ('orange', 4), ('red', 1), ('orange', 4)]
d = defaultdict(set)
for k, v in s:
    d[k].add(v)
print(d.items())

dict_items([('red', {1, 3}), ('orange', {2, 4})])


In [29]:
from collections import OrderedDict

# regular unsorted dictionary
d = {'banana': 3, 'apple': 4, 'pear': 1, 'orange': 2}

# dictionary sorted by key
print(OrderedDict(sorted(d.items(), key=lambda t: t[0])))

# dictionary sorted by value
print(OrderedDict(sorted(d.items(), key=lambda t: t[1])))

# dictionary sorted by length of the key string
print(OrderedDict(sorted(d.items(), key=lambda t: len(t[0]))))

OrderedDict([('apple', 4), ('banana', 3), ('orange', 2), ('pear', 1)])
OrderedDict([('pear', 1), ('orange', 2), ('banana', 3), ('apple', 4)])
OrderedDict([('pear', 1), ('apple', 4), ('banana', 3), ('orange', 2)])
